# Calculate Embeddings

Ajuda a calcular embeddings dos dados adquiridos dos comentários das redes sociais de políticos.

Entrada: `Post-filtrado.xlsx`

Saída: `Embeddings_(NOME_DO_MODELO).xlsx`

In [87]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer

BASE_PATH = 'dados/'
DIRECTORY = BASE_PATH + 'preprocessed/embeddings/'

Iniciação das variáveis que iremos utilizar e das colunas na qual iremos trabalhar


In [88]:
# 1. Pegar o modelo para testar
TYPE_MODEL = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
model = SentenceTransformer(TYPE_MODEL)

# 2. Pegar as sentenças (nesse caso, no Post-filtrado)
file_path = BASE_PATH + 'Post-filtrado.xlsx'
file_path_features = 'Embeddings_all-MiniLM-paraphrase-multilingual.xlsx'
#é bom modificar o nome manualmente, pois pode dar erro na questão do save

column_text = "Texto"
column_id = "ID"
column_author = "Autor"
column_likes = "Curtidas"

In [89]:
# 3. Ler os Arquivos e Remover NaN
rf = pd.read_excel(file_path)
rf = rf.dropna(subset=[column_text])

rf.head(3)

,ID,Autor,Data,Texto,Link,Rede,Tipo,Curtidas,Comentários,Compart.,Hora
0,7114971700365692165,Jair Bolsonaro,2022-06-30,#jairbolsonaro #bolsonaro #palavras #president...,https://www.tiktok.com/@bolsonaromessiasjair/v...,tiktok,Video,24400,1083,1566,NaN
1,7115033431473474822,Lula,2022-06-30,Alô alô geração tiktoker! Imagina só um #gover...,https://www.tiktok.com/@lulaoficial/video/7115...,tiktok,Video,11700,809,589,NaN
2,7115050482179050758,Jair Bolsonaro,2022-06-30,#emprego #jair #bolsonaro #jairbolsonaro #pres...,https://www.tiktok.com/@bolsonaromessiasjair/v...,tiktok,Video,9163,480,1140,NaN


Função para verificar se o texto contém apenas hashtags

Caso o comentário só possua hashtags ele irá retornar  `true`, caso contrário a função retorna `false`

In [90]:
def contains_only_hashtags(text):
    hashtags = re.findall(r'#\S+', text)
    return len(hashtags) == len(text.split())

In [91]:
ids = rf[column_id].tolist()
authors = rf[column_author].tolist()
sentences = rf[column_text].tolist()
likes = rf[column_likes].tolist()

Encode dos embeddings utilizando a library do "sentence-transformers" e também a concatenação de dados previamente informados da planilha passada como `ID`,`Candidato` e `Curtidas`

In [92]:
# 4. Calcular os embeddings das sentenças
embeddings = model.encode(sentences)
df_embeddings = pd.DataFrame(embeddings) 
df_embeddings.columns = [f'x{i+1}' for i in range(df_embeddings.shape[1])]

df_final = pd.DataFrame({
    column_id: ids,
    "Candidato": authors,
    column_likes: likes
})

df_final['Only Hashtags'] = rf[column_text].apply(contains_only_hashtags)

df_final = pd.concat([df_final, df_embeddings], axis=1)

A planilha ficará com as colunas importantes originais (__ID__,__Candidato__ e __Curtidas__), e também estará com uma flag que mostra se há hashtags(__Only Hashtags__), e por fim, features calculadas pelo embeddinG   __x1__,__x2__,__x3__ ... __xN__ , onde N seria o tamanho de dimensões que aquele modelo possui

In [93]:
df_final.head(5)

,ID,Candidato,Curtidas,Only Hashtags,x1,x2,x3,x4,x5,x6,...,x375,x376,x377,x378,x379,x380,x381,x382,x383,x384
0,7114971700365692165,Jair Bolsonaro,24400,True,-0.035405,0.091901,0.029305,-0.098718,0.243411,0.111185,...,0.175389,-0.227796,-0.141506,-0.010863,0.124115,0.060907,0.403370,-0.108669,-0.133018,0.144618
1,7115033431473474822,Lula,11700,False,-0.222600,0.258170,0.165599,-0.210579,0.359918,-0.001123,...,0.370087,-0.112563,-0.277257,0.095219,-0.293415,0.101862,0.038032,0.054848,0.059788,0.128184
2,7115050482179050758,Jair Bolsonaro,9163,True,0.024929,0.079934,0.098492,-0.100231,0.224104,0.025696,...,0.168720,-0.311732,-0.105087,-0.059519,0.052269,0.108190,0.341778,-0.142647,-0.096181,0.125113
3,7115120078982630661,Jair Bolsonaro,3485,True,0.037256,0.072214,-0.000965,-0.107916,0.288683,0.103231,...,0.202172,-0.211109,-0.122603,0.045138,0.120379,0.205129,0.261554,-0.083097,-0.185110,0.220620
4,7115161088219565317,Jair Bolsonaro,22100,True,0.184212,-0.063834,-0.014328,-0.073712,0.067048,-0.046129,...,0.109674,-0.212620,-0.229578,-0.191411,0.090878,0.120460,0.276041,-0.130134,-0.236629,0.108136


In [94]:
# 5. Salvar no Excel

df_final.to_excel((DIRECTORY + file_path_features), index=False)